In [3]:
#import tensorflow as tf
import os
import numpy as np
import cv2  
import glob 
from matplotlib import pyplot as plt
import joblib

In [4]:
# for tensorflow only
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [5]:
#saving model 
def save_mod(model,Model_name):
    change_dir(r"C:\Users\iamne\JupyterNotebookServer\dev\ProjAlzhiemer")
    Model_name+='.sav'
    joblib.dump(model,Model_name)

In [6]:
#Changing Current Working Directory;
path = r"E:\Dataset\Alzheimer_s Dataset"
def change_dir(path):
    retval = os.getcwd()    # Check current working directory.
    print("Current working directory ", retval) 
    os.chdir( path )   # Now change the directory.
    retval = os.getcwd() # Check current working directory.
    print ("Directory changed successfully ",retval) 
change_dir(path)

Current working directory  C:\Users\iamne\JupyterNotebookServer\dev\ProjAlzhiemer
Directory changed successfully  E:\Dataset\Alzheimer_s Dataset


In [7]:
# reading images from directory

def load_image(path):
    lst=[]
    for image_path in glob.glob(path):
        img = cv2.imread(image_path,0)
        img = img.reshape(1,36608)
        lst.append(img[0])
    return lst
    
temp = path + r"\train\MildDemented\*.jpg"
Mild_test = load_image(temp)
print("Mild = ",len(Mild_test))
temp = path + r"\train\ModerateDemented\*.jpg"
Mod_test = load_image(temp)
print("Mod = ",len(Mod_test))
temp = path + r"\train\NonDemented\*.jpg"
Non_test = load_image(temp)
print("Non = ",len(Non_test))
temp = path + r"\train\VeryMildDemented\*.jpg"
VMild_test = load_image(temp)
print("VMild = ",len(VMild_test))

Mild =  717
Mod =  52
Non =  2560
VMild =  1792


In [8]:
# Verifying image loading
#plt.axis("off")
#plot = plt.imshow(Mild_test[178])

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier


In [10]:
train_x = Mild_test+VMild_test+Mod_test+Non_test
train_y = []

def ohe_ml(*lst):
    train_y = []
    en =1
    for j in range(0,len(lst)):
        for i in range(0,len(lst[j])):
            train_y.append(en)
        en*=10
    return train_y
train_y= ohe_ml( Mild_test,VMild_test,Mod_test,Non_test)
    

In [11]:
print(len(train_x),len(train_y))
print(train_x[0].shape)
loc = 732
print(train_x[loc],train_y[loc])

5121 5121
(36608,)
[0 0 0 ... 0 0 0] 10


In [12]:
models = list()

decision_tree = Pipeline([('m', DecisionTreeClassifier())])
models.append(('decision', decision_tree))

randomforest = Pipeline([('m', RandomForestClassifier())])
models.append(('randomforest', randomforest))

svc = Pipeline([('m', SVC())])
models.append(('svc', svc))
# define the voting ensemble
ensemble = VotingClassifier(estimators=models, voting='hard',n_jobs=-1)

In [11]:
models

[('decision', Pipeline(steps=[('m', DecisionTreeClassifier())])),
 ('randomforest', Pipeline(steps=[('m', RandomForestClassifier())])),
 ('svc', Pipeline(steps=[('m', SVC())]))]

In [12]:
ensemble

VotingClassifier(estimators=[('decision',
                              Pipeline(steps=[('m',
                                               DecisionTreeClassifier())])),
                             ('randomforest',
                              Pipeline(steps=[('m',
                                               RandomForestClassifier())])),
                             ('svc', Pipeline(steps=[('m', SVC())]))],
                 n_jobs=-1)

In [13]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
n_scores = cross_val_score(ensemble, train_x, train_y, scoring='accuracy', cv=cv, n_jobs=6)

In [14]:
#n_scores.mean()


In [15]:
e = ensemble.fit(train_x,train_y)

In [16]:
save_mod(e,"test_tf_df")

Current working directory  E:\Dataset\Alzheimer_s Dataset
Directory changed successfully  C:\Users\iamne\JupyterNotebookServer\dev\ProjAlzhiemer
